In [105]:
import pandas as pd
import os
from datetime import datetime

In [106]:
# -- years being used for evaluation
Years = ['2014,2015,2013&2014&2015']

# -- spatial key being used for calculation
SpatialKey = 'ZIPCODE'

# -- use Fiscal or Calendar year
UseFiscalYear = False
if UseFiscalYear == True:
    YearType = 'FiscalYr'
else:
    YearType = 'CalendarYr'
# -- input path
ParkDataInputPath = '../../shapeData/Tables/ParkQualityScores/'

# -- output path
WriteOutputPath = '../../shapeData/Tables/AllAttributes/'

In [107]:
# -- Bring In external Data
if SpatialKey == 'ZIPCODE':
    # -- Set Date
    Date = '07-13-2015'
    # -- Input Path
    DataInputPath = '../../shapeData/Tables/ZipcodeExternalData/'
    # -- Bring in the socioeconomic Data
    ZipcodesocioEconomicData = pd.read_csv(DataInputPath+'ZipCodeExternalDataSummary.csv')
    # -- Bring in the market Value Data
    medianMarketValueZipCode = pd.read_csv(DataInputPath+'medianMarketValueZipCode.csv')
    medianMarketValueZipCode = medianMarketValueZipCode[['ZipCode','Market_Val']]
    # -- Bring in the economic Data & clean
    economicDataZipCode = pd.read_csv(DataInputPath+'zbp13totals.csv')
    economicDataZipCode = economicDataZipCode[['zip','emp','ap','est']]
    economicDataZipCode = economicDataZipCode.rename(columns={'zip':'ZipCode','emp':'TotalMidMarchEmployees', 'ap':'TotalAnnualPayroll($1,000)', 'est':'TotalNumberofEstablishments'})
    # -- Mereg All data
    ZipCodeMarketValueSocioeconomic = pd.merge(ZipcodesocioEconomicData, medianMarketValueZipCode, right_on='ZipCode', left_on= 'ZipCode5', how='left')
    AllExternalData = pd.merge(ZipCodeMarketValueSocioeconomic, economicDataZipCode, on= 'ZipCode', how='left')
    AllExternalData['ZIPCODE']=AllExternalData['ZipCode5']
else:
    # -- Set Date
    Date = '07-14-2015'
    # -- Input Path
    DataInputPath = '../../shapeData/Tables/CensusTractExternalData/'
    # -- Bring in the socioeconomic Data
    censustractsocioEconomicData = pd.read_csv(DataInputPath+'CensusTractExternalDataSummary.csv')
    censustractsocioEconomicData['GEOID'] = censustractsocioEconomicData['GEOID'].astype(str)
    # -- Bring in the market Value Data
    medianMarketValueCensusTract = pd.read_csv(DataInputPath+'medianMarketValueCensusTract.csv')
    medianMarketValueCensusTract = medianMarketValueCensusTract[['CensusTract','Market_Val']]
    # -- Bring in the employess Data 
    employeeCensusTract = pd.read_csv(DataInputPath+'WACemployeesCensusTract2012.csv')
    employeeCensusTract = employeeCensusTract[['GEOID','NumberOfEmployeesWAC']]
    employeeCensusTract['GEOID'] = employeeCensusTract['GEOID'].astype(str)
    # -- Merge All data
    CensusTractMarketValueSocioeconomic = pd.merge( medianMarketValueCensusTract,censustractsocioEconomicData, left_on='CensusTract', right_on= 'GEOID', how='left')
    AllExternalData = pd.merge( CensusTractMarketValueSocioeconomic,employeeCensusTract, on = 'GEOID', how = 'left')

In [108]:
def ReadScores(Year, Date):
# -- Bring in Park Quality Data and Merge
    if UseFiscalYear:
        parkQuality = pd.read_csv(InputPath + 'ParkQuality_%s_%s_%s_%s.csv' % (SpatialKey, Year, YearType, Date), index_col=0)
        if SpatialKey == 'GEOID':
            parkQuality['GEOID'] = parkQuality['GEOID'].astype(str)
    else:
        parkQuality = pd.read_csv(InputPath + 'ParkQuality_%s_%s_%s_%s.csv' % (SpatialKey, Year, YearType, Date), index_col=0)
        if SpatialKey == 'GEOID':
            parkQuality['GEOID'] = parkQuality['GEOID'].astype(str)

    return pd.merge(AllExternalData, parkQuality, on= SpatialKey, how='left')

In [109]:
def WriteScores(df, Year):
    # -- Write to csv
    now = datetime.now()
    if UseFiscalYear:
        YearType = 'FiscalYr'
    else:
        YearType = 'CalendarYr'
    df.to_csv(WriteOutputPath + 'AllAttributes_%s_%s_%s_%s.csv' % ( SpatialKey, Year.replace(',','&'), YearType, now.strftime('%m-%d-%Y')), sep=',')

In [110]:
for Year in Years:
    AllAttributes = ReadScores(Year, Date)
    WriteScores(AllAttributes, Year)

In [ ]:
AllAttributes